In [ ]:
import pandas as pd
import os 
import json
import matplotlib.pyplot as plt
import nilearn.plotting as niplot
import ipywidgets as widgets
import nibabel as nib
import numpy as np
import plotly.graph_objects as go
from ipywidgets import IntSlider, VBox, HBox, Output
from IPython.display import display
from nilearn.image import resample_to_img
from nilearn.image import smooth_img
from nilearn.masking import apply_mask, unmask
from scipy.stats import pearsonr
import matplotlib.gridspec as gridspec
from nilearn.interfaces.fmriprep import load_confounds
from nilearn.signal import clean

Reference article for imaging protocol and experimen design: 10.1093/cercor/bhz157

In [ ]:


def interactive_fmri_viewer(img, mask=None):
    """
    Interactive viewer for 4D fMRI data (axial slices over time), with optional mask overlay.

    Parameters:
    -----------
    img : nibabel.Nifti1Image
        Loaded 4D fMRI image.

    mask : nibabel.Nifti1Image or None
        Optional 3D or 4D binary mask image to overlay. Must be in same space as img.

    Usage:
    ------
    >>> import nibabel as nib
    >>> img = nib.load("peer_run1.nii.gz")
    >>> mask = nib.load("eye_mask_right_resampled.nii.gz")  # 3D or 4D mask
    >>> interactive_fmri_viewer(img, mask)
    """
    data = img.get_fdata()
    x, y, z, t = data.shape

    mask_data = None
    if mask is not None:
        mask_data = mask.get_fdata().astype(bool)
        # Check mask dimension
        if mask_data.ndim == 3:
            is_4d_mask = False
        elif mask_data.ndim == 4:
            if mask_data.shape[3] != t:
                raise ValueError(f"Mask time dimension ({mask_data.shape[3]}) does not match image ({t})")
            is_4d_mask = True
        else:
            raise ValueError("Mask must be either 3D or 4D")

    @widgets.interact(
        z=widgets.IntSlider(min=0, max=z - 1, step=1, value=z // 2, description="Axial Slice"),
        t=widgets.IntSlider(min=0, max=t - 1, step=1, value=0, description="Time Point")
    )
    def show_fmri_slice(z=0, t=0):
        plt.figure(figsize=(6, 6))
        plt.imshow(data[:, :, z, t].T, cmap="gray", origin="lower")

        if mask_data is not None:
            if is_4d_mask:
                mask_slice = mask_data[:, :, z, t]
            else:
                mask_slice = mask_data[:, :, z]
            # Overlay the mask in red with alpha blending
            masked_slice = np.ma.masked_where(~mask_slice, mask_slice)
            plt.imshow(masked_slice.T, cmap='Reds', alpha=0.4, origin="lower")

        plt.title(f"Time: {t}, Slice: {z}")
        plt.axis("off")
        plt.show()


def correlation_with_gaze_task(func_img, mask=None, axis=None):
   
    # Apply mask
    masked_data = apply_mask(func_img, mask)  # shape (timepoints, voxels)

    # Get regressor
    regressor = stimuli_repeated[axis].values  # shape (timepoints,)

    # Compute voxelwise correlations
    correlations = np.array([
        pearsonr(masked_data[:, v], regressor)[0]
        for v in range(masked_data.shape[1])
    ])

    # Reconstruct 3D correlation map
    correlation_img = unmask(correlations, mask)

    return correlation_img

from scipy.ndimage import center_of_mass
from tqdm import tqdm  # For progress bar

def compute_weighted_centers(img_4d, top10_mask_4d):
    """
    Computes weighted center of mass for each time point using top 10% voxel mask.

    Parameters:
    -----------
    img_4d : nibabel.Nifti1Image
        4D fMRI image.

    top10_mask_4d : nibabel.Nifti1Image
        4D binary mask image indicating top 10% voxels for each time point.

    Returns:
    --------
    centers : np.ndarray of shape (T, 3)
        Weighted center of mass (x, y, z) at each time point.
    """
    data = img_4d.get_fdata()
    mask = top10_mask_4d.get_fdata().astype(bool)
    T = data.shape[3]

    centers = []
    for t in tqdm(range(T), desc="Computing weighted centers"):
        vol = data[..., t]
        mask_t = mask[..., t]

        if np.sum(mask_t) == 0:
            centers.append((np.nan, np.nan, np.nan))
            continue

        # Use voxel intensities within the mask as weights
        weights = vol * mask_t
        com = center_of_mass(weights)
        centers.append(com)

    return np.array(centers)


def compute_unweighted_centers(img_4d, top10_mask_4d):
    """
    Computes unweighted (geometric) center of mass for each time point using top 10% voxel mask.

    Parameters:
    -----------
    img_4d : nibabel.Nifti1Image
        4D fMRI image (only used to get time dimension shape).

    top10_mask_4d : nibabel.Nifti1Image
        4D binary mask image indicating top 10% voxels for each time point.

    Returns:
    --------
    centers : np.ndarray of shape (T, 3)
        Geometric center of mass (x, y, z) at each time point.
    """
    mask = top10_mask_4d.get_fdata().astype(bool)
    T = mask.shape[3]

    centers = []
    for t in tqdm(range(T), desc="Computing unweighted centers"):
        mask_t = mask[..., t]

        if np.sum(mask_t) == 0:
            centers.append((np.nan, np.nan, np.nan))
            continue

        # Unweighted center of mass: center_of_mass of a binary mask
        com = center_of_mass(mask_t)
        centers.append(com)

    return np.array(centers)

In [ ]:
bids_dir=('/media/koba/MULTIBOOT/peer/HBN_BIDS/')

eye_mask_right_name=os.path.join(bids_dir,'source/nau_mask_right.nii.gz')
eye_mask_right=nib.load(eye_mask_right_name)
eye_mask_left_name=os.path.join(bids_dir,'source/nau_mask_left.nii.gz')
eye_mask_left=nib.load(eye_mask_left_name)

peer_run1_name=os.path.join(bids_dir,'derivatives/sub-NDARCT889DMB/ses-HBNsiteCBIC/func/sub-NDARCT889DMB_ses-HBNsiteCBIC_task-peer_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
peer_run1_confounds=pd.read_csv(os.path.join(bids_dir,'derivatives/sub-NDARCT889DMB/ses-HBNsiteCBIC/func/sub-NDARCT889DMB_ses-HBNsiteCBIC_task-peer_run-1_desc-confounds_timeseries.tsv'), delimiter='\t')
peer_run3_name=os.path.join(bids_dir,'derivatives/sub-NDARCT889DMB/ses-HBNsiteCBIC/func/sub-NDARCT889DMB_ses-HBNsiteCBIC_task-peer_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')

peer_run1=nib.load(peer_run1_name)

json_functional='/media/koba/MULTIBOOT/peer/HBN_BIDS/sub-NDARCT889DMB/ses-HBNsiteCBIC/func/sub-NDARCT889DMB_ses-HBNsiteCBIC_task-peer_run-1_bold.json'

# Load and display JSON
with open(json_functional, 'r') as f:
    data = json.load(f)

#print(json.dumps(data, indent=4))

print("Shape:", peer_run1.shape)

In [ ]:

# Load eye positions via task file 
stimuli_values = pd.read_csv(os.path.join(bids_dir, 'source/stim_vals.csv'))

# Repeat each point to match the fMRI's resolution
stimuli_repeated = stimuli_values.loc[stimuli_values.index.repeat(5)].reset_index(drop=True)

# Set up a custom gridspec layout: 2 rows, 2 columns, with the right column merged
fig = plt.figure(figsize=(12, 4))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1])

# Subplot for pos_x time series (top left)
ax0 = fig.add_subplot(gs[0, 0])
stimuli_repeated.pos_x.plot.line(ax=ax0, label='pos_x', color='blue')
ax0.set_ylabel('Position X')
ax0.set_ylim(-1.2, 1.2)
ax0.set_title("pos_x over time")

# Subplot for pos_y time series (bottom left)
ax1 = fig.add_subplot(gs[1, 0])
stimuli_repeated.pos_y.plot.line(ax=ax1, label='pos_y', color='blue')
ax1.set_xlabel('Time point')
ax1.set_ylabel('Position Y')
ax1.set_ylim(-1.2, 1.2)
ax1.set_title("pos_y over time")

# Subplot for pos_x vs pos_y scatter plot (right, spanning both rows)
ax2 = fig.add_subplot(gs[:, 1])
ax2.scatter(stimuli_repeated.pos_x, stimuli_repeated.pos_y, color='blue', s=5, alpha=0.7)
ax2.set_xlabel('Position X')
ax2.set_ylabel('Position Y')
ax2.set_xlim(-1.2, 1.2)
ax2.set_ylim(-1.2, 1.2)
ax2.set_title("pos_x vs pos_y")

plt.tight_layout()
plt.show()

In [ ]:
# Resample the masks to the resolution of the functional image
eye_mask_right_resampled = resample_to_img(
    source_img=eye_mask_right,       # 3D mask to resample
    target_img=peer_run1,            # 4D target image (spatial dims used)
    interpolation='nearest'          # Use nearest neighbor to preserve binary values
)

eye_mask_left_resampled = resample_to_img(
    source_img=eye_mask_left,       # 3D mask to resample
    target_img=peer_run1,            # 4D target image (spatial dims used)
    interpolation='nearest'          # Use nearest neighbor to preserve binary values
)
print("Mask shape before resampling:", eye_mask_left.shape)
print("Mask shape after resampling:", eye_mask_left_resampled.shape)
print("Shape of the functional scan:", peer_run1.shape)

In [ ]:
interactive_fmri_viewer(peer_run1, mask=eye_mask_right_resampled)

In [ ]:
# Smooth the image
peer_run1_smoothed = smooth_img(peer_run1, fwhm=4)
interactive_fmri_viewer(peer_run1_smoothed, mask=eye_mask_left_resampled,)

 Functional data was corrected with field-map sequence <br>
 No slice-time correction was applied <br>
 Motion correction should be the first (and probably the only one that everyone would agree on) preprocessing step 

In [ ]:
correlation_y_right=correlation_with_gaze_task(peer_run1_smoothed, mask=eye_mask_right_resampled, axis="pos_y")
correlation_y_left=correlation_with_gaze_task(peer_run1_smoothed, mask=eye_mask_left_resampled, axis="pos_y")
correlation_y = nib.Nifti1Image((correlation_y_right.get_fdata()+correlation_y_left.get_fdata()),
                                affine=correlation_y_right.affine, header=correlation_y_right.header)

correlation_x_right=correlation_with_gaze_task(peer_run1_smoothed, mask=eye_mask_right_resampled, axis="pos_x")
correlation_x_left=correlation_with_gaze_task(peer_run1_smoothed, mask=eye_mask_left_resampled, axis="pos_x")
correlation_x = nib.Nifti1Image((correlation_x_right.get_fdata()+correlation_x_left.get_fdata()),
                                affine=correlation_x_right.affine, header=correlation_x_right.header)

In [ ]:
niplot.view_img(correlation_y, title='Y axis',draw_cross=False, bg_img='/home/koba/fsl/data/standard/MNI152_T1_1mm.nii.gz')

In [ ]:
niplot.view_img(correlation_x, title='Y axis',draw_cross=True, bg_img='/home/koba/fsl/data/standard/MNI152_T1_1mm.nii.gz')

In [ ]:
# Get the data array
min_len=135
peer_data=peer_run1_smoothed.get_fdata()
data_x=correlation_x.get_fdata()
max_idx = np.unravel_index(np.argmax(data_x), data_x.shape)
max_val = data_x[max_idx]
print(f"Max correlation value with x: {max_val:.4f} at voxel location (x, y, z): {max_idx}")

voxel_ts=peer_data[max_idx[0],max_idx[1],max_idx[2],:]
voxel_ts_norm = (voxel_ts - voxel_ts.min()) / (voxel_ts.max() - voxel_ts.min())  # [0,1]
voxel_ts_rescaled_x = voxel_ts_norm * 2 - 1  # scale to [-1,1]

data_y=correlation_y.get_fdata()
max_idx = np.unravel_index(np.argmax(data_y), data_y.shape)
max_val = data_y[max_idx]
voxel_ts=peer_data[max_idx[0],max_idx[1],max_idx[2],:]
voxel_ts_norm = (voxel_ts - voxel_ts.min()) / (voxel_ts.max() - voxel_ts.min())  # [0,1]
voxel_ts_rescaled_y = voxel_ts_norm * 2 - 1  # scale to [-1,1]

print(f"Max correlation value with y: {max_val:.4f} at voxel location (x, y, z): {max_idx}")

# Rescale stimulus pos_x
stim_x = stimuli_repeated.pos_x.iloc[:min_len].values
stim_x_norm = (stim_x - stim_x.min()) / (stim_x.max() - stim_x.min())
stim_x_rescaled = stim_x_norm * 2 - 1

# Rescale stimulus pos_y
stim_y = stimuli_repeated.pos_y.iloc[:min_len].values
stim_y_norm = (stim_y - stim_y.min()) / (stim_y.max() - stim_y.min())
stim_y_rescaled = stim_y_norm * 2 - 1

# Set up the grid
fig = plt.figure(figsize=(12, 4))
gs = gridspec.GridSpec(2, 2)

# Top-left: stimulus X vs eye center X
ax0 = fig.add_subplot(gs[0, 0])
ax0.plot(stim_x_rescaled, label='stim_x', color='blue')
ax0.plot(voxel_ts_rescaled_x , label='eye_x', color='orange', alpha=0.7)
ax0.set_title('X Coordinate Comparison')
ax0.legend(loc='lower right')

# Bottom-left: stimulus Y vs eye center Y
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(stim_y_rescaled, label='stim_y', color='blue')
ax1.plot(voxel_ts_rescaled_y , label='eye_y', color='green', alpha=0.7)
ax1.set_title('Y Coordinate Comparison')
ax1.legend(loc='lower right')

# Right: scatter plot (stimulus vs eye)
ax2 = fig.add_subplot(gs[:, 1])
ax2.scatter(stim_x_rescaled, stim_y_rescaled, label='stimulus', color='blue', s=10, alpha=0.5)
ax2.scatter(voxel_ts_rescaled_x, voxel_ts_rescaled_y , label='eye', color='orange', s=10, alpha=0.5)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_xlim(-1.2, 1.2)
ax2.set_ylim(-1.2, 1.2)
ax2.set_title("Stimulus vs Eye Trajectory")
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:

# 1. Load data from image (shape: X, Y, Z, T)
data_4d = peer_run1.get_fdata()
original_shape = data_4d.shape  # (x, y, z, t)
n_voxels = np.prod(original_shape[:-1])
n_timepoints = original_shape[-1]

# 2. Reshape to 2D: (voxels, time)
data_2d = data_4d.reshape(-1, n_timepoints).T  # Shape: (T, voxels)

# 3. Prepare confounds (remove NaNs/infs)
confounds_clean = peer_run1_confounds[["white_matter", "csf", "framewise_displacement", "global_signal","trans_x","trans_y","trans_z","rot_x","rot_y","rot_z"]].copy()
confounds_clean = confounds_clean.replace([np.inf, -np.inf], np.nan)
confounds_clean = confounds_clean.fillna(confounds_clean.mean())
confounds_array = confounds_clean.values

# 4. Apply clean: regress confounds, detrend, standardize, and bandpass
TR = 0.8  # Replace with actual TR in seconds if different
cleaned_2d = clean(
    data_2d,
    confounds=None,
    detrend=False,
    standardize="zscore_sample",
    low_pass=0.04,
    high_pass=0.01,
    t_r=TR
)

# 5. Reshape back to 4D
cleaned_4d = cleaned_2d.T.reshape(original_shape)

# 6. Create cleaned Nifti image
peer1_clean_img = nib.Nifti1Image(cleaned_4d, affine=peer_run1_smoothed.affine, header=peer_run1_smoothed.header)


#interactive_fmri_viewer(peer1_clean_img)
peer1_clean_img = smooth_img(peer1_clean_img, fwhm=4)
interactive_fmri_viewer(peer1_clean_img, mask=eye_mask_left_resampled,)

correlation_y_right=correlation_with_gaze_task(peer1_clean_img, mask=eye_mask_right_resampled, axis="pos_y")
correlation_y_left=correlation_with_gaze_task(peer1_clean_img, mask=eye_mask_left_resampled, axis="pos_y")
correlation_y = nib.Nifti1Image((correlation_y_right.get_fdata()+correlation_y_left.get_fdata()),
                                affine=correlation_y_right.affine, header=correlation_y_right.header)

correlation_x_right=correlation_with_gaze_task(peer1_clean_img, mask=eye_mask_right_resampled, axis="pos_x")
correlation_x_left=correlation_with_gaze_task(peer1_clean_img, mask=eye_mask_left_resampled, axis="pos_x")
correlation_x = nib.Nifti1Image((correlation_x_right.get_fdata()+correlation_x_left.get_fdata()),
                                affine=correlation_x_right.affine, header=correlation_x_right.header)

In [ ]:
niplot.view_img(correlation_y, title='Y axis',draw_cross=False, bg_img='/home/koba/fsl/data/standard/MNI152_T1_1mm.nii.gz')

In [ ]:
niplot.view_img(correlation_x, title='Y axis',draw_cross=False, bg_img='/home/koba/fsl/data/standard/MNI152_T1_1mm.nii.gz')

In [ ]:

def top_10_percent_mask(masked_data):
    """
    For each time point, create a mask selecting voxels in the top 10% brightness.
    
    Parameters:
    masked_data : np.ndarray
        Array of shape (timepoints, voxels)
        
    Returns:
    mask : np.ndarray (bool)
        Boolean array of shape (timepoints, voxels) where True indicates top 10% voxels.
    """
    mask = np.zeros_like(masked_data, dtype=bool)
    
    for t in range(masked_data.shape[0]):
        data_t = masked_data[t]
        threshold = np.percentile(data_t, 10)  # top 10% threshold
        mask[t] = data_t >= threshold
    
    return mask

masked_data=apply_mask(peer_run1_smoothed,eye_mask_right_resampled)
#masked_img=nib.Nifti1Image(masked_data, affine=peer_run1_smoothed.affine, header=peer_run1_smoothed.header)

# Usage
top10_mask = top_10_percent_mask(masked_data)  # shape: (135, 4177)


# top10_mask shape: (135, 4177) - timepoints × voxels
# unmask expects shape (n_samples, n_features), so transpose to (4177, 135)
top10_mask_T = top10_mask.astype(float)  # convert bool to float (1.0/0.0)

# Unmask returns a 4D image (x, y, z, time)
top10_mask_img = unmask(top10_mask_T, eye_mask_right_resampled)

# Save if needed

In [ ]:
# remove the cluster mask from eye mask to leave optic nerve 

In [ ]:
interactive_fmri_viewer(peer_run1_smoothed,mask=top10_mask_img)

In [ ]:
centers = compute_weighted_centers(peer_run1_smoothed, top10_mask_img)

In [ ]:
# Clean input
centers_valid = centers[~np.isnan(centers).any(axis=1)]
pos_x_valid = stimuli_repeated.pos_x.iloc[:len(centers_valid)]
pos_y_valid = stimuli_repeated.pos_y.iloc[:len(centers_valid)]

# Store both r and p
correlations = {
    'x_vs_pos_x': pearsonr(centers_valid[:, 0], pos_x_valid),
    'y_vs_pos_x': pearsonr(centers_valid[:, 1], pos_x_valid),
    'z_vs_pos_x': pearsonr(centers_valid[:, 2], pos_x_valid),
    'x_vs_pos_y': pearsonr(centers_valid[:, 0], pos_y_valid),
    'y_vs_pos_y': pearsonr(centers_valid[:, 1], pos_y_valid),
    'z_vs_pos_y': pearsonr(centers_valid[:, 2], pos_y_valid),
}

# Print results
for k, (r, p) in correlations.items():
    print(f"{k}: r = {r:.4f}, p = {p:.4g}")


In [ ]:

# Ensure same length
min_len = min(len(centers_valid), len(stimuli_repeated))

# Rescale eye center X (from centers_valid[:, 0])
voxel_ts_x = voxel_ts_rescaled_x_left#centers_valid[:min_len, 0]
voxel_ts_x_norm = (voxel_ts_x - voxel_ts_x.min()) / (voxel_ts_x.max() - voxel_ts_x.min())
voxel_ts_rescaled_x = voxel_ts_x_norm * 2 - 1

# Rescale eye center Z (from centers_valid[:, 2], used as vertical/Y)
voxel_ts_z = voxel_ts_rescaled_y_left#centers_valid[:min_len, 2]
voxel_ts_z_norm = (voxel_ts_z - voxel_ts_z.min()) / (voxel_ts_z.max() - voxel_ts_z.min())
voxel_ts_rescaled_y = voxel_ts_z_norm * 2 - 1

# Rescale stimulus pos_x
stim_x = voxel_ts_rescaled_x_right#stimuli_repeated.pos_x.iloc[:min_len].values
stim_x_norm = (stim_x - stim_x.min()) / (stim_x.max() - stim_x.min())
stim_x_rescaled = stim_x_norm * 2 - 1

# Rescale stimulus pos_y
stim_y = voxel_ts_rescaled_y_right *-1#stimuli_repeated.pos_y.iloc[:min_len].values
stim_y_norm = (stim_y - stim_y.min()) / (stim_y.max() - stim_y.min())
stim_y_rescaled = stim_y_norm * 2 - 1

# Set up the grid
fig = plt.figure(figsize=(12, 4))
gs = gridspec.GridSpec(2, 2)

# Top-left: stimulus X vs eye center X
ax0 = fig.add_subplot(gs[0, 0])
ax0.plot(stim_x_rescaled, label='stim_x', color='blue')
ax0.plot(voxel_ts_rescaled_x * -1, label='eye_x', color='orange', alpha=0.7)
ax0.set_title('X Coordinate Comparison')
ax0.legend(loc='lower right')

# Bottom-left: stimulus Y vs eye center Y
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(stim_y_rescaled, label='stim_y', color='blue')
ax1.plot(voxel_ts_rescaled_y * -1, label='eye_y', color='green', alpha=0.7)
ax1.set_title('Y Coordinate Comparison')
ax1.legend(loc='lower right')

# Right: scatter plot (stimulus vs eye)
ax2 = fig.add_subplot(gs[:, 1])
ax2.scatter(stim_x_rescaled, stim_y_rescaled, label='stimulus', color='blue', s=10, alpha=0.5)
ax2.scatter(voxel_ts_rescaled_x * -1, voxel_ts_rescaled_y * -1, label='eye', color='orange', s=10, alpha=0.5)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_xlim(-1.2, 1.2)
ax2.set_ylim(-1.2, 1.2)
ax2.set_title("Stimulus vs Eye Trajectory")
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
voxel_ts_rescaled_x_right=voxel_ts_rescaled_x
voxel_ts_rescaled_y_right=voxel_ts_rescaled_y

In [ ]:
voxel_ts_rescaled_x_left=voxel_ts_rescaled_x
voxel_ts_rescaled_y_left=voxel_ts_rescaled_y

In [ ]:
pearsonr(voxel_ts_rescaled_x_right, voxel_ts_rescaled_x_left)

# Print correlation coefficient and p-value


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Compute correlation
r, p = pearsonr(voxel_ts_rescaled_y_right, voxel_ts_rescaled_y_left)

# Create scatter plot with regression line and confidence interval
plt.figure(figsize=(4, 4))
sns.regplot(
    x=voxel_ts_rescaled_y_right,
    y=voxel_ts_rescaled_y_left,
    ci=95,
    scatter_kws={'alpha': 0.6, 's': 40},
    line_kws={'color': 'red'}
)
plt.title(f"Right vs. Left Eye X (rescaled)\nr = {r:.4f}, p = {p:.2e}")
plt.xlabel("Right Eye X (rescaled)")
plt.ylabel("Left Eye X (rescaled)")
plt.grid(True)
plt.axis("equal")
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import pearsonr
import pandas as pd
import numpy as np

# Ensure confounds_clean is a DataFrame with column names
if isinstance(confounds_clean, np.ndarray):
    confounds_clean = pd.DataFrame(confounds_clean, columns=[f"confound_{i}" for i in range(confounds_clean.shape[1])])

# Combine centers and confounds into one DataFrame
combined = pd.concat([
    pd.DataFrame(centers_valid, columns=["x", "y", "z"]),
    confounds_clean.reset_index(drop=True)
], axis=1)

# Drop rows with NaNs or Infs
combined = combined.replace([np.inf, -np.inf], np.nan).dropna()

# Now split back
centers_clean = combined[["x", "y", "z"]].values
confounds_cleaned = combined.drop(columns=["x", "y", "z"])

# Compute correlations
results = []
for i, coord_label in enumerate(["x", "y", "z"]):
    for conf_name in confounds_cleaned.columns:
        r, p = pearsonr(centers_clean[:, i], confounds_cleaned[conf_name].values)
        results.append({
            "center_axis": coord_label,
            "confound": conf_name,
            "r": r,
            "p": p
        })

# Format and show results
correlation_df = pd.DataFrame(results)
correlation_df_sorted = correlation_df.reindex(correlation_df.r.abs().sort_values(ascending=False).index)

print(correlation_df_sorted.head(10))


In [ ]:
print(correlation_df_stim_sorted.head(10))


In [ ]:
correlation_df_stim

In [ ]:
from scipy.stats import pearsonr
import pandas as pd
import numpy as np

# Ensure confounds_clean is a DataFrame with column names
if isinstance(confounds_clean, np.ndarray):
    confounds_clean = pd.DataFrame(confounds_clean, columns=[f"confound_{i}" for i in range(confounds_clean.shape[1])])

# Ensure stimuli_repeated has exactly 2 columns: pos_x and pos_y
stimuli_df = stimuli_repeated[["pos_x", "pos_y"]].copy()
stimuli_df.columns = ["x", "y"]  # Rename to align with rest of code

# Add dummy z = 0 (since your original loop expects x, y, z)
stimuli_df["z"] = 0

# Combine with confounds
combined = pd.concat([
    stimuli_df.reset_index(drop=True),
    confounds_clean.reset_index(drop=True)
], axis=1)

# Drop rows with NaNs or Infs
combined = combined.replace([np.inf, -np.inf], np.nan).dropna()

# Split
stimuli_clean = combined[["x", "y", "z"]].values
confounds_cleaned = combined.drop(columns=["x", "y", "z"])

# Compute correlations
results = []
for i, coord_label in enumerate(["x", "y", "z"]):
    for conf_name in confounds_cleaned.columns:
        r, p = pearsonr(stimuli_clean[:, i], confounds_cleaned[conf_name].values)
        results.append({
            "stim_axis": coord_label,
            "confound": conf_name,
            "r": r,
            "p": p
        })

# Format and show
correlation_df_stim = pd.DataFrame(results)
correlation_df_stim_sorted = correlation_df_stim.reindex(correlation_df.r.abs().sort_values(ascending=False).index)

print(correlation_df_stim_sorted.head(10))


In [ ]:
pearsonr(correlation_df_stim.iloc[:19].r,correlation_df.iloc[:19].r)

In [ ]:
from scipy.signal import decimate, welch
import numpy as np
import matplotlib.pyplot as plt

def estimate_nyquist_rate(signal, fs):
    # Welch's method to estimate power spectrum
    freqs, power = welch(signal, fs=fs)
    # Estimate cutoff frequency where 95% of power is contained
    cumulative_power = np.cumsum(power) / np.sum(power)
    cutoff_idx = np.searchsorted(cumulative_power, 0.95)
    f_max = freqs[cutoff_idx]
    nyquist_rate = 2 * f_max
    return nyquist_rate, f_max, freqs, power

# Example: suppose your original stimulus sampling rate is 60 Hz
fs_original = 0.8  # Hz
nyquist_rate, f_max, freqs, power = estimate_nyquist_rate(stim_x, fs=fs_original)

# Plot power spectrum
plt.figure(figsize=(8, 3))
plt.semilogy(freqs, power)
plt.axvline(f_max, color='r', linestyle='--', label=f"95% Power Cutoff = {f_max:.2f} Hz")
plt.title("Stim_x Power Spectrum (Welch)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.legend()
plt.tight_layout()
plt.show()

# Downsample if needed
factor = int(np.floor(fs_original / nyquist_rate)) if nyquist_rate > 0 else 1
stim_x_resampled = decimate(stim_x, factor, ftype='fir') if factor > 1 else stim_x
stim_y_resampled = decimate(stim_y, factor, ftype='fir') if factor > 1 else stim_y


In [ ]:
from scipy.signal import decimate

# Downsample centers[:, 0] with same factor
if nyquist_rate > 0:
    downsample_factor = int(np.floor(fs_original / nyquist_rate))
else:
    downsample_factor = 1  # fallback

# Ensure factor is at least 1
downsample_factor = max(downsample_factor, 1)

# Apply FIR decimation to centers[:, 0]
centers_x_downsampled = decimate(centers[:, 0], downsample_factor, ftype='fir') if downsample_factor > 1 else centers[:, 0]


In [ ]:
plt.plot(centers_x_downsampled)
plt.

# MVPA

In [ ]:
peer_run1_smoothed.shape

In [ ]:
from nilearn.masking import apply_mask
from nilearn.plotting import plot_matrix
import matplotlib.pyplot as plt
import numpy as np

# Apply the mask to extract voxel time series
# Returns shape: (n_timepoints, n_voxels)
masked_data = apply_mask(peer_run1_smoothed, eye_mask_left_resampled)

# Transpose to shape: (n_voxels, n_timepoints)
masked_data = masked_data

# Optionally z-score across time to normalize
from scipy.stats import zscore
masked_data = zscore(masked_data, axis=1)

# Compute correlation matrix across voxels
correlation_matrix = np.corrcoef(masked_data)

# Plot correlation matrix
plot_matrix(correlation_matrix, figure=(8, 6),
            labels=None, auto_fit=True,
            title='Voxel-wise Correlation (Left Eye Region)')
plt.show()


In [ ]:
correlation_matrix[:,0].shape

In [ ]:

voxel_ts=smoothed_signal
voxel_ts_norm = (voxel_ts - voxel_ts.min()) / (voxel_ts.max() - voxel_ts.min())  # [0,1]
voxel_ts_rescaled_x = voxel_ts_norm * 2 - 1  # scale to [-1,1]

data_y=correlation_y.get_fdata()
max_idx = np.unravel_index(np.argmax(data_y), data_y.shape)
max_val = data_y[max_idx]
voxel_ts=smoothed_signal
voxel_ts_norm = (voxel_ts - voxel_ts.min()) / (voxel_ts.max() - voxel_ts.min())  # [0,1]
voxel_ts_rescaled_y = voxel_ts_norm * 2 - 1  # scale to [-1,1]

print(f"Max correlation value with y: {max_val:.4f} at voxel location (x, y, z): {max_idx}")

# Rescale stimulus pos_x
stim_x = stimuli_repeated.pos_x.iloc[:min_len].values
stim_x_norm = (stim_x - stim_x.min()) / (stim_x.max() - stim_x.min())
stim_x_rescaled = stim_x_norm * 2 - 1

# Rescale stimulus pos_y
stim_y = stimuli_repeated.pos_y.iloc[:min_len].values
stim_y_norm = (stim_y - stim_y.min()) / (stim_y.max() - stim_y.min())
stim_y_rescaled = stim_y_norm * 2 - 1

# Set up the grid
fig = plt.figure(figsize=(12, 4))
gs = gridspec.GridSpec(2, 2)

# Top-left: stimulus X vs eye center X
ax0 = fig.add_subplot(gs[0, 0])
ax0.plot(stim_x_rescaled, label='stim_x', color='blue')
ax0.plot(voxel_ts_rescaled_x , label='eye_x', color='orange', alpha=0.7)
ax0.set_title('X Coordinate Comparison')
ax0.legend(loc='lower right')

# Bottom-left: stimulus Y vs eye center Y
ax1 = fig.add_subplot(gs[1, 0])
ax1.plot(stim_y_rescaled, label='stim_y', color='blue')
ax1.plot(voxel_ts_rescaled_y , label='eye_y', color='green', alpha=0.7)
ax1.set_title('Y Coordinate Comparison')
ax1.legend(loc='lower right')

# Right: scatter plot (stimulus vs eye)
ax2 = fig.add_subplot(gs[:, 1])
ax2.scatter(stim_x_rescaled, stim_y_rescaled, label='stimulus', color='blue', s=10, alpha=0.5)
ax2.scatter(voxel_ts_rescaled_x, voxel_ts_rescaled_y , label='eye', color='orange', s=10, alpha=0.5)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_xlim(-1.2, 1.2)
ax2.set_ylim(-1.2, 1.2)
ax2.set_title("Stimulus vs Eye Trajectory")
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# Original signal
signal = correlation_matrix[:, 0]

# Apply Savitzky-Golay filter (window size must be odd and < len(signal))
smoothed_signal = savgol_filter(signal, window_length=11, polyorder=3)

# Plot original and smoothed
plt.figure(figsize=(10, 4))
plt.plot(signal, label="Original", alpha=0.5)
plt.plot(smoothed_signal, label="Smoothed (Savitzky-Golay)", linewidth=2)
plt.title("Denoising with Savitzky-Golay Filter")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
left_mask_data = eye_mask_left_resampled.get_fdata().astype(bool)

# Get shape
x, y, z = left_mask_data.shape

# Define y-splits (3 equal parts)
y1 = y // 3
y2 = 2 * y // 3

# Create three sub-masks
mask_left_part1 = np.zeros_like(left_mask_data)
mask_left_part2 = np.zeros_like(left_mask_data)
mask_left_part3 = np.zeros_like(left_mask_data)

# Apply original mask within y segments
mask_left_part1[:, :y1, :] = left_mask_data[:, :y1, :]
mask_left_part2[:, y1:y2, :] = left_mask_data[:, y1:y2, :]
mask_left_part3[:, 10:, :] = left_mask_data[:, 10:, :]

# Convert back to NIfTI images using the same affine
mask_img1 = nib.Nifti1Image(mask_left_part1.astype(np.uint8), affine=eye_mask_left_resampled.affine)
mask_img2 = nib.Nifti1Image(mask_left_part2.astype(np.uint8), affine=eye_mask_left_resampled.affine)
mask_img3 = nib.Nifti1Image(mask_left_part3.astype(np.uint8), affine=eye_mask_left_resampled.affine)


In [ ]:
niplot.view_img(mask_img3,bg_img='/home/koba/fsl/data/standard/MNI152_T1_1mm.nii.gz')

In [ ]:
interactive_fmri_viewer(smooth_img, mask=mask_img1)